In [1]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

from skimage.filters import threshold_otsu
from skimage.morphology import closing, square

In [2]:
video_path = "../video/rilevamento-intrusioni-video.wm"
cap = cv2.VideoCapture(video_path)

assert cap.isOpened(), "Not opened!"

fps = int(cap.get(cv2.CAP_PROP_FPS))
length = cap.get(cv2.CAP_PROP_FRAME_COUNT) / fps

print(f"[I] Video FPS: {fps}")
print(f"[I] Video Length: {length}")

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"[I] Video Frame Width: {width}")
print(f"[I] Video Frame height: {height}")

[I] Video FPS: 12
[I] Video Length: 41.916666666666664
[I] Video Frame Width: 320
[I] Video Frame height: 240


In [3]:
# MOG2 Background Subtractor
mog2 = cv2.createBackgroundSubtractorMOG2(history=fps * 2, varThreshold=20, detectShadows=False)
#mog2.setNMixtures(1)

In [8]:
# """
#     Performed a sorted insertion of pv into the frame buffer in the pi row.
#     pv: pixel value
#     pi: pixel index
#     frame_buffer (global)
# """
# def sortAndInsert(pv, pi, ):
#     global frame_buffer
#     insertion_idx = np.searchsorted(frame_buffer[pi], pv)
#     if insertion_idx == len(frame_buffer[pi]):
#         insertion_idx =- 1

#     frame_buffer[pi, insertion_idx] = pv

#cv2.imshow("frame_copy", frame_contours)
# largestContour = max(contours, key = cv2.contourArea) # get largest contour
# rect = cv2.minAreaRect(largestContour)
# box = np.int64(cv2.boxPoints(rect))
# cv2.drawContours(frame_contours, [box], 0, (0,0,255), 1)
# cv2.imshow("frame_contours", frame_contours)

# cv2.imshow(f"temporalMedianBackground-frame-{frame_count}", temporalMedianBackground)
# cmd = cv2.waitKey(0)
# cv2.destroyWindow(f"temporalMedianBackground-frame-{frame_count}")
# if cmd == ord("q"):
#     break
# if cmd == ord("n"):
#     continue

# alpha = 3.0 # Contrast control
# beta = 25 # Brightness control
# # call convertScaleAbs function
# #adjusted = cv2.convertScaleAbs(mog2_fgmask, alpha=alpha, beta=beta)


# plt.hist(frame_diff.ravel(), 256, [0, 256])
# plt.show()


            
# Disegna la bounding box
#cv2.rectangle(frame_contours, (x_min, y_min), (x_max, y_max), 255, 2)

#cv2.imshow("frame_contours", frame_contours)


# analysis = cv2.connectedComponentsWithStats(tmb_fgmask, 8, cv2.CV_32S)
# (totalLabels, label_ids, values, centroid) = analysis 

# # Initialize a new image to store  
# # all the output components 
# output = np.zeros(tmb_fgmask.shape, dtype="uint8") 

# # Loop through each component 
# for i in range(1, totalLabels): 
    
#     # Area of the component 
#     area = values[i, cv2.CC_STAT_AREA]  
    
#     if (area > 140) and (area < 400): 
#         componentMask = (label_ids == i).astype("uint8") * 255
#         output = cv2.bitwise_or(output, componentMask) 

# cv2.imshow("Filtered Components", output) 


In [5]:
def preprocessing(frame):
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Apply Median Blur
    median = cv2.medianBlur(gray, 3)
    gaussian = cv2.GaussianBlur(median, (3, 3), 0)
    return gaussian

In [6]:
# # Set up the window for fullscreen display
# window_name = "Fullscreen Quad View"
# cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
# cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

In [7]:
# HYPERPARAMTERS
MAX_HISTORY = 4 * fps
SKIP_FRAMES = fps // 2
THRESHOLD = 25

# 320x240 rows, 60 columns
# frame_buffer = np.empty((width * height,MAX_HISTORY), dtype=np.uint8)

start = time.time()
frames = []
initialized = False
frame_count = 0
skip_count = 0
fgmask_diff = None
temporalMedianBackground = None

while(cap.isOpened()):
    ret, frame_original = cap.read()
    if not ret or frame_original is None:
        cap.release()
        print("Released Video Resource")
        break

    frame = preprocessing(frame_original)
    frame_count += 1
    skip_count += 1

    mog2_fgmask = mog2.apply(frame)
    #mog2_fgmask = cv2.erode(mog2_fgmask,  np.ones((3, 3), np.uint8), iterations=1)
    mog2_fgmask = cv2.medianBlur(mog2_fgmask, 3)
    #cv2.imshow("Mog2 foreground mask", mog2_fgmask)

    # # tentativo
    blur = cv2.GaussianBlur(mog2_fgmask, (5,5), 0)
    thresh = threshold_otsu(blur)
    if thresh > 0:    
        bw = closing(blur >= thresh, square(7))
        erode = cv2.medianBlur(bw.astype(np.uint8) * 255, 3)
        dilated = cv2.dilate(erode, None, iterations=5)

        frame_contours = frame.copy()
        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        real_countours = []
        for contour in contours:
            (x, y, w, h) = cv2.boundingRect(contour)
            
            if cv2.contourArea(contour) > 100:
                real_countours.append(contour)
            
        # evitare che impazzisca se non ci sono bboxes
        if len(real_countours) > 0:
            # Calcola la bounding box
            # Define the ROI            
            x_min = min([np.min(cnt[:, :, 0]) for cnt in real_countours])
            x_max = max([np.max(cnt[:, :, 0]) for cnt in real_countours])
            y_min = min([np.min(cnt[:, :, 1]) for cnt in real_countours])
            y_max = max([np.max(cnt[:, :, 1]) for cnt in real_countours])

    if initialized:
        # print(f"[I] Skipping Frames... {skip_count}")
        if skip_count > SKIP_FRAMES:
            skip_count = 0
            frames.pop(0)
            frames.append(frame)
            # print(f"[I] Added Image...")

            temporalMedianBackground = np.median(frames, axis=0).astype(dtype=np.uint8)
            
    if not initialized and len(frames) < MAX_HISTORY:
        # print(f"[I] Learning...")
        frames.append(frame)
        
    elif not initialized and len(frames) == MAX_HISTORY:
        # print(f"Compute First Estimate")
        # compute the first background estimation
        initialized = True    
        temporalMedianBackground = np.median(frames, axis=0).astype(dtype=np.uint8)

    # Compute the AbsDiff between temporalMedianBackground and the Current Frame
    if initialized:
        frame_diff = cv2.absdiff(temporalMedianBackground, frame)
        threshold, tmb_fgmask = cv2.threshold(frame_diff, THRESHOLD, 255, cv2.THRESH_BINARY)       # cv2.adaptiveThreshold
        
        # logical or between mog2 and temporalMedianBackground to create the shape of the fast movement using ROI
        fgmask = mog2_fgmask.copy()
        fgmask[y_min:y_max, x_min:x_max] = cv2.bitwise_or(mog2_fgmask[y_min:y_max, x_min:x_max], tmb_fgmask[y_min:y_max, x_min:x_max])    
        #cv2.imshow("Bitwise Mask", fgmask)
        
        kernel = np.ones((3,3),np.uint8)
        closing_img = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel, iterations=3)
        #cv2.imshow("closing", closing_img)

        # override frozen values
        if fgmask_diff is not None:
            tmb_fgmask = cv2.bitwise_or(tmb_fgmask, fgmask_diff)
            cv2.imshow("tmb_fgmask", tmb_fgmask)
            # TODO: contours and areas

        # static objects
        fgmask_diff = cv2.bitwise_and(tmb_fgmask, cv2.bitwise_not(closing_img))
        #cv2.imshow("fgmask_diff", fgmask_diff)

        cmd = cv2.waitKey(0)
        if cmd == ord("q"):
            break
        if cmd == ord("n"):
            continue
        

end = time.time()

print(f"[I] Elapsed time: {end - start} s")
    
cap.release()
cv2.destroyAllWindows()


/tmp/ipykernel_19762/3159033442.py:37: FutureWarning: `square` is deprecated since version 0.25 and will be removed in version 0.27. Use `skimage.morphology.footprint_rectangle` instead.
  bw = closing(blur >= thresh, square(7))


Released Video Resource
[I] Elapsed time: 32.64481735229492 s
